In [1]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras import layers
print (tf.VERSION)
print (tf.keras.__version__)

1.13.1
2.2.4-tf


In [2]:
import numpy as np
import pandas as pd
import os

path_folder=os.path.normcase(r'C:\Users\dell\Downloads\Titanic-maching-learning-from-disaster')
train_data_origin=pd.read_csv(os.path.join(path_folder,'train.csv'),sep=',')
test_data_origin=pd.read_csv(os.path.join(path_folder,'test.csv'),sep=',')

Data preprocessing

In [4]:
def deal_ticket_fare(df):
    '''
    计算每张票有几人共享，以及每人花费的船票钱
    
    paramters:
        df--dataframe，待处理的数据表
        
    return:
        df_count--处理后，添加新列的数据表
    '''
    num_of_tickets=df[['Ticket']].groupby(df['Ticket']).count()
    num_of_tickets.columns=['num_of_tickets']
    df_count=df.merge(num_of_tickets,left_on='Ticket',right_index=True,how='left')
    df_count['fare_per_ticket']=df_count['Fare']/df_count['num_of_tickets']
    return df_count

cols=['Sex_T','Embarked_T','Age','Fare','fare_per_ticket','num_of_tickets','Pclass','SibSp','Parch']
train_data_count=deal_ticket_fare(train_data_origin)
trans_sex={'male':0,'female':1}
trans_embarked={'S':0,'C':1,'Q':2}

train_data_count['Sex_T']=train_data_count['Sex'].map(trans_sex)
#from sklearn.impute import MissingIndicator
#indicator=MissingIndicator(missing_values=np.nan)
#train_missing_indicator=indicator.fit_transform(train_data_count)
from sklearn.impute import SimpleImputer
imp1=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
train_data_count['Embarked']=imp1.fit_transform(train_data_count[['Embarked']])
imp2=SimpleImputer(missing_values=np.nan,strategy='mean')
train_data_count['Age']=imp2.fit_transform(train_data_count[['Age']])
train_data_count['Embarked_T']=train_data_count['Embarked'].map(trans_embarked).astype(np.int)
cols=['Sex_T','Embarked_T','Age','Fare','fare_per_ticket','num_of_tickets','Pclass','SibSp','Parch']
#X_train_data=train_data_count.reindex(columns=cols).values
#y_train_data=train_data_count['Survived'].ravel()
from sklearn.preprocessing import  OneHotEncoder
enc=OneHotEncoder(handle_unknown='ignore')
enc_train_features=enc.fit_transform(train_data_count[['Sex_T','Embarked_T']]).toarray()
from sklearn.preprocessing import StandardScaler
std_scaler=StandardScaler()
sca_train_features=std_scaler.fit_transform(train_data_count[['Age','Fare','fare_per_ticket','num_of_tickets','Pclass','SibSp','Parch']].values.astype(np.float64))
#sca_features.shape
train_data_X=np.c_[enc_train_features,sca_train_features]
train_data_y=train_data_count['Survived'].values.ravel()

In [5]:
values_dict={'Fare':0.0}
test_data_origin.fillna(value=values_dict,inplace=True)
test_data_count=deal_ticket_fare(test_data_origin)
test_data_count['Sex_T']=test_data_count['Sex'].map(trans_sex)
test_data_count['Embarked']=imp1.transform(test_data_count[['Embarked']])
test_data_count['Embarked_T']=test_data_count['Embarked'].map(trans_embarked).astype(np.int)
test_data_count['Age']=imp2.transform(test_data_count[['Age']])
enc_test_features=enc.fit_transform(test_data_count[['Sex_T','Embarked_T']]).toarray()
sca_test_features=std_scaler.fit_transform(test_data_count[['Age','Fare','fare_per_ticket',\
                                                              'num_of_tickets','Pclass','SibSp','Parch']].values.astype(np.float64))
test_data_X=np.c_[enc_test_features,sca_test_features]

Model

In [6]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(10,kernel_regularizer=tf.keras.regularizers.l1(0.01)))
model.add(tf.keras.layers.Dense(7,activation='relu'))
model.add(tf.keras.layers.Dense(7,kernel_regularizer=tf.keras.regularizers.l1(0.01)))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [7]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),loss='binary_crossentropy',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.fit(train_data_X,train_data_y,epochs=100,batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
891/891 [==============================] - 1s 678us/sample - loss: 1.1109 - acc: 0.5915
Epoch 2/100
891/891 [==============================] - 0s 41us/sample - loss: 1.0250 - acc: 0.6700
Epoch 3/100
891/891 [==============================] - 0s 49us/sample - loss: 0.9783 - acc: 0.6790
Epoch 4/100
891/891 [==============================] - 0s 40us/sample - loss: 0.9410 - acc: 0.6835
Epoch 5/100
891/891 [==============================] - 0s 41us/sample - loss: 0.9051 - acc: 0.7048
Epoch 6/100
891/891 [==============================] - 0s 41us/sample - loss: 0.8687 - acc: 0.7116
Epoch 7/100
891/891 [==============================] - 0s 40us/sample - loss: 0.8337 - acc: 0.7250
Epoch 8/100
891/891 [==============================] - 0s 46us/sample - loss: 0.8012 - acc: 0.7329
Epoch 9/100
891/891 [==============================] - 0s 48us/sample - loss: 0.7668 - acc: 0.7419
Epoch 10/100
891/891 [==============================] - 0s 4

891/891 [==============================] - 0s 38us/sample - loss: 0.4447 - acc: 0.8316
Epoch 81/100
891/891 [==============================] - 0s 39us/sample - loss: 0.4439 - acc: 0.8272
Epoch 82/100
891/891 [==============================] - 0s 43us/sample - loss: 0.4446 - acc: 0.8215
Epoch 83/100
891/891 [==============================] - 0s 43us/sample - loss: 0.4435 - acc: 0.8328
Epoch 84/100
891/891 [==============================] - 0s 47us/sample - loss: 0.4431 - acc: 0.8283
Epoch 85/100
891/891 [==============================] - 0s 43us/sample - loss: 0.4413 - acc: 0.8294
Epoch 86/100
891/891 [==============================] - 0s 39us/sample - loss: 0.4418 - acc: 0.8283
Epoch 87/100
891/891 [==============================] - 0s 39us/sample - loss: 0.4422 - acc: 0.8272
Epoch 88/100
891/891 [==============================] - 0s 40us/sample - loss: 0.4412 - acc: 0.8305
Epoch 89/100
891/891 [==============================] - 0s 39us/sample - loss: 0.4410 - acc: 0.8316
Epoch 90/100


In [9]:
test_y_pred_prob=model.predict(test_data_X).ravel()

In [10]:
y_test_pred=(test_y_pred_prob>0.5).astype(np.int)
test_data_count['Survived']=y_test_pred
test_data_count[['PassengerId','Survived']].to_csv('c:/users/dell/desktop/gender_submission.csv',index=False)